In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 123 kB 5.6 MB/s 
     |████████████████████████████████| 132 kB 54.2 MB/s 
     |████████████████████████████████| 8.6 MB 25.1 MB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 
     |████████████████████████████████| 138 kB 77.0 MB/s 
     |████████████████████████████████| 127 kB 74.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 28.7 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 895 kB 63.6 MB/s 
     |████████████████████████████████| 596 kB 68.2 MB/s 
     |████████████████████████████████| 6.5 MB 60.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import spacy
from tqdm import tqdm
from pytorch_pretrained_bert.optimization import BertAdam
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
from torch.utils.data import Dataset, TensorDataset, RandomSampler, SequentialSampler, DataLoader

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_set = pd.read_csv('/content/drive/Shareddrives/NLP/ASBA/data/traindata.csv',sep='\t',names=['polarity','aspect','target','offset','sentence'])

In [ ]:
val_set = pd.read_csv('/content/drive/Shareddrives/NLP/ASBA/data/devdata.csv',sep='\t',names=['polarity','aspect','target','offset','sentence'])

In [ ]:
train_set['polarity'].value_counts(normalize = True)

positive    0.701929
negative    0.259481
neutral     0.038589
Name: polarity, dtype: float64

In [ ]:
train_set.head(5)

,polarity,aspect,target,offset,sentence
0,positive,AMBIENCE#GENERAL,seating,18:25,short and sweet – seating is great:it's romant...
1,positive,AMBIENCE#GENERAL,trattoria,25:34,This quaint and romantic trattoria is at the t...
2,positive,FOOD#QUALITY,food,98:102,The have over 100 different beers to offer thi...
3,negative,SERVICE#GENERAL,STAFF,5:10,THIS STAFF SHOULD BE FIRED.
4,positive,FOOD#STYLE_OPTIONS,menu,4:8,"The menu looked great, and the waiter was very..."


In [ ]:
print('length of train set',len(train_set))
print('length of test set',len(val_set))


length of train set 1503
length of test set 376


# BERT-based

## Original data (Please ignore)

In [ ]:
# def parse_SemEval14(fn):
#     root=ET.parse(fn).getroot()
#     corpus=[]
#     opin_cnt=[0]*len(polar_idx)
#     for sent in root.iter("sentence"):
#         opins=set()
#         for opin in sent.iter('aspectTerm'):
#             if int(opin.attrib['from'] )!=int(opin.attrib['to'] ) and opin.attrib['term']!="NULL":
#                 if opin.attrib['polarity'] in polar_idx:
#                     opins.add((opin.attrib['term'], int(opin.attrib['from']), int(opin.attrib['to']), opin.attrib['polarity'] ) )
#         for ix, opin in enumerate(opins):
#             opin_cnt[polar_idx[opin[3] ] ]+=1
#             corpus.append({"id": sent.attrib['id']+"_"+str(ix), "sentence": sent.find('text').text, "term": opin[0], "polarity": opin[-1]})
#     print (opin_cnt)
#     print (len(corpus))
#     return corpus

In [ ]:
# results = parse_SemEval14('Restaurants_Train.xml')

[2164, 807, 637]
3608


In [ ]:
# results[0]

{'id': '3121_0',
 'sentence': 'But the staff was so horrible to us.',
 'term': 'staff',
 'polarity': 'negative'}

## Useful functions

In [ ]:
from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("activebus/BERT-XD_Review")

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [ ]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


In [ ]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s.""" #check later if we can merge this function with the SQuAD preprocessing 
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a =tokenizer.tokenize(example.text_a)
        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)]

        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        
        label_id = label_map[example.label]

        features.append(
                InputFeatures(
                        input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id))
    return features


In [ ]:
def _create_examples(dataset, set_type):
        """Creates examples for the training and dev sets."""
        """
        Parameters:
        dataset:
        set_type:'train'/'test'
        '"""
        examples = []
        for index, row in dataset.iterrows():
            guid = "%s-%s" % (set_type, index )
            text_a = row['target']
            text_b = row['sentence']
            label = row['polarity']
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples  

In [ ]:
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x

In [ ]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

## Model training

In [ ]:
def train(dataset, batch_size = 12, epochs=5, max_seq_length = 138, learning_rate = 3e-5,warmup_proportion=0.1):
#     processor = data_utils.AscProcessor()
    label_list = ["positive", "negative", "neutral"]
    train_examples = _create_examples(train_set,'train')
    num_train_steps = int(len(train_examples) / batch_size) * epochs

    train_features = convert_examples_to_features(
        train_examples, label_list, max_seq_length, tokenizer)
#     logger.info("***** Running training *****")
#     logger.info("  Num examples = %d", len(train_examples))
#     logger.info("  Batch size = %d", args.train_batch_size)
#     logger.info("  Num steps = %d", num_train_steps)
    
    all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)

    all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

    all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)

    all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

    
    train_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    
#     #>>>>> validation
#     if args.do_valid:
#         valid_examples = processor.get_dev_examples(args.data_dir)
#         valid_features=data_utils.convert_examples_to_features(
#             valid_examples, label_list, args.max_seq_length, tokenizer, "asc")
#         valid_all_input_ids = torch.tensor([f.input_ids for f in valid_features], dtype=torch.long)
#         valid_all_segment_ids = torch.tensor([f.segment_ids for f in valid_features], dtype=torch.long)
#         valid_all_input_mask = torch.tensor([f.input_mask for f in valid_features], dtype=torch.long)
#         valid_all_label_ids = torch.tensor([f.label_id for f in valid_features], dtype=torch.long)
#         valid_data = TensorDataset(valid_all_input_ids, valid_all_segment_ids, valid_all_input_mask, valid_all_label_ids)

#         logger.info("***** Running validations *****")
#         logger.info("  Num orig examples = %d", len(valid_examples))
#         logger.info("  Num split examples = %d", len(valid_features))
#         logger.info("  Batch size = %d", args.train_batch_size)

#         valid_sampler = SequentialSampler(valid_data)
#         valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=args.train_batch_size)    

#         best_valid_loss=float('inf')
#         valid_losses=[]
#     #<<<<< end of validation declaration

#     model = AutoModel.from_pretrained("activebus/BERT-XD_Review")
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = len(label_list) )
    model.to(device)
    # Prepare optimizer
    param_optimizer = [(k, v) for k, v in model.named_parameters() if v.requires_grad==True]
    param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    t_total = num_train_steps
    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=learning_rate,
                         warmup=warmup_proportion,
                         t_total=t_total)

    global_step = 0
    model.train()
    for _ in range(epochs):
        for step, batch in enumerate(tqdm(train_dataloader)):
            batch = tuple(t.to(device) for t in batch)
            input_ids, segment_ids, input_mask, label_ids = batch
            loss = model(input_ids, segment_ids, input_mask, label_ids)
            loss.backward()

            lr_this_step = learning_rate * warmup_linear(global_step/t_total,warmup_proportion)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_this_step
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
#             #>>>> perform validation at the end of each epoch .
#         if args.do_valid:
#             model.eval()
#             with torch.no_grad():
#                 losses=[]
#                 valid_size=0
#                 for step, batch in enumerate(valid_dataloader):
#                     batch = tuple(t.cuda() for t in batch) # multi-gpu does scattering it-self
#                     input_ids, segment_ids, input_mask, label_ids = batch
#                     loss = model(input_ids, segment_ids, input_mask, label_ids)
#                     losses.append(loss.data.item()*input_ids.size(0) )
#                     valid_size+=input_ids.size(0)
#                 valid_loss=sum(losses)/valid_size
#                 logger.info("validation loss: %f", valid_loss)
#                 valid_losses.append(valid_loss)
#             if valid_loss<best_valid_loss:
#                 torch.save(model, os.path.join(args.output_dir, "model.pt") )
#                 best_valid_loss=valid_loss
#             model.train()
#     if args.do_valid:
#         with open(os.path.join(args.output_dir, "valid.json"), "w") as fw:
#             json.dump({"valid_losses": valid_losses}, fw)
#     else:
    torch.save(model,  "model.pt" )




In [ ]:
train(train_set)

  0%|          | 0/126 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
100%|██████████| 126/126 [00:47<00:00,  2.65it/s]


## Model evaluation

In [ ]:
def test(dataset,max_seq_length=128, batch_size=6):  
    # Load a trained model that you have fine-tuned (we assume evaluate on cpu)    

    label_list = ["positive", "negative", "neutral"]
    eval_examples = _create_examples(dataset,'test')
    
#     tokenizer = BertTokenizer.from_pretrained(modelconfig.MODEL_ARCHIVE_MAP[args.bert_model])

    eval_features = convert_examples_to_features(eval_examples, label_list, max_seq_length, tokenizer)


    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_segment_ids, all_input_mask, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=batch_size)

    model = torch.load("model.pt")
    model.to(device)
    model.eval()
    
    full_logits=[]
    predicted_labels=[]
    full_label_ids=[]
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = tuple(t.to(device) for t in batch)
#         batch = tuple(t.cuda() for t in batch)
        input_ids, segment_ids, input_mask, label_ids = batch
        
        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.cpu().numpy()

        full_logits.extend(logits.tolist() )
        full_label_ids.extend(label_ids.tolist() )
        predicted_labels = np.argmax(full_logits, axis=1)

    return predicted_labels, full_label_ids



In [ ]:
# Evaluation
def load_label_output(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        return [line.strip().split("\t")[0] for line in f if line.strip()]

def eval_list(glabels, slabels):
    if (len(glabels) != len(slabels)):
        print("\nWARNING: label count in system output (%d) is different from gold label count (%d)\n" % (
        len(slabels), len(glabels)))
    n = min(len(slabels), len(glabels))
    incorrect_count = 0
    for i in range(n):
        if slabels[i] != glabels[i]: incorrect_count += 1
    acc = (n - incorrect_count) / n
    return acc*100


In [ ]:
slabel_ids,glabel_ids = test(val_set)

100%|██████████| 63/63 [00:03<00:00, 19.35it/s]


In [ ]:
label_dict = {0:'positive',1:'negative',2:'neutral'}
slabels = [label_dict[i] for i in slabel_ids]

In [ ]:
print('the accuracy of predictions:', format(eval_list(glabel_ids,slabel_ids),'.2f'),'%')

the accuracy of predictions: 83.51 %


In [ ]:
# Reference: https://aclanthology.org/W19-6120.pdf
# Pre-processing entity and aspect pairs for BERT

# In order to fit better the BERT model when training and to be able have the pretrained data in BERT to be applicable, 
# we formatted it to have a sentence-like structure, so the pair ”FOOD#STYLE OPTIONS” gets parsed into ”food, style options”. 
# This text is what we use as aspect.



In [ ]:
# Fix class imbalance

# To compensate for the imbalance, we weight each label depending on how frequently they show up in the training set. 
# The higher the frequency of a label, the lower the weight of the given label.



# Sentiment terms

In [ ]:
# Get the sentiment terms

sentiment_terms = []
for review in nlp.pipe(train_set['sentence']):
        if review.is_parsed:
            sentiment_terms.append(' '.join([token.lemma_ for token in review if (not token.is_stop and not token.is_punct and (token.pos_ == "ADJ" or token.pos_ == "VERB"))]))
        else:
            sentiment_terms.append('')  
train_set['sentiment_terms'] = sentiment_terms
train_set.head(10)

In [ ]:
# Build the sentiment model
sentiment_model = Sequential()
sentiment_model.add(Dense(512, input_shape=(6000,), activation='relu'))
sentiment_model.add(Dense(3, activation='softmax'))
sentiment_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

sentiment_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(dataset.sentiment_terms))

label_encoder_2 = LabelEncoder()
integer_sentiment = label_encoder_2.fit_transform(dataset.sentiment)
dummy_sentiment = to_categorical(integer_sentiment)

sentiment_model.fit(sentiment_tokenized, dummy_sentiment, epochs=5, verbose=1)